In [ ]:
!pip install investpy


import urllib.request, json , time, os, difflib, itertools
import pandas as pd
from multiprocessing.dummy import Pool
from datetime import datetime
from datetime import date
import dateutil
from datetime import timezone
import time
import pathlib
import numpy as np
pd.options.mode.chained_assignment = None

try:
    import httplib
except:
    import http.client as httplib

In [ ]:
def check_internet():
    conn = httplib.HTTPConnection("www.google.com", timeout=5)
    try:
        conn.request("HEAD", "/")
        conn.close()
        print("connected")
        return True
    except:
        conn.close()
        print("not connected")
        return False

Functions for parabolic SAR https://en.wikipedia.org/wiki/Parabolic_SAR (Source https://github.com/je-suis-tm/quant-trading)

In [ ]:
def signal_generation(df,method):
        new=method(df)
        new['positions'],new['signals']=0,0
        new['positions']=np.where(new['real sar']<new['Close'],1,0)
        new['signals']=new['positions'].diff()
        return new
def parabolic_sar(new):
    
    #this is common accelerating factors for forex and commodity
    #for equity, af for each step could be set to 0.01
    initial_af=0.02
    step_af=0.02
    end_af=0.2
    
    
    new['trend']=0
    new['sar']=0.0
    new['real sar']=0.0
    new['ep']=0.0
    new['af']=0.0

    #initial values for recursive calculation
    new['trend'][1]=1 if new['Close'][1]>new['Close'][0] else -1
    new['sar'][1]=new['High'][0] if new['trend'][1]>0 else new['Low'][0]
    new.at[1,'real sar']=new['sar'][1]
    new['ep'][1]=new['High'][1] if new['trend'][1]>0 else new['Low'][1]
    new['af'][1]=initial_af

    #calculation
    for i in range(2,len(new)):
        
        temp=new['sar'][i-1]+new['af'][i-1]*(new['ep'][i-1]-new['sar'][i-1])
        if new['trend'][i-1]<0:
            new.at[i,'sar']=max(temp,new['High'][i-1],new['High'][i-2])
            temp=1 if new['sar'][i]<new['High'][i] else new['trend'][i-1]-1
        else:
            new.at[i,'sar']=min(temp,new['Low'][i-1],new['Low'][i-2])
            temp=-1 if new['sar'][i]>new['Low'][i] else new['trend'][i-1]+1
        new.at[i,'trend']=temp
    
        
        if new['trend'][i]<0:
            temp=min(new['Low'][i],new['ep'][i-1]) if new['trend'][i]!=-1 else new['Low'][i]
        else:
            temp=max(new['High'][i],new['ep'][i-1]) if new['trend'][i]!=1 else new['High'][i]
        new.at[i,'ep']=temp
    
    
        if np.abs(new['trend'][i])==1:
            temp=new['ep'][i-1]
            new.at[i,'af']=initial_af
        else:
            temp=new['sar'][i]
            if new['ep'][i]==new['ep'][i-1]:
                new.at[i,'af']=new['af'][i-1]
            else:
                new.at[i,'af']=min(end_af,new['af'][i-1]+step_af)
        new.at[i,'real sar']=temp
       
        
    return new

#Pull data from http://investing.com
The code in the following cell pulls market data of the symbols in the dictionary "symbols_and_names". You could configure the following variables


1.   number_of_months -> Length of the history.
2.   rows, cols, height, width -> these variables relevant to the charts. You could configure them according to your screen size.








In [77]:
import investpy
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import plot
import plotly
import math
from datetime import datetime, timedelta
from dateutil import relativedelta
import operator

def draw_line_plot(data,title,y_axis,row,col,fig):
    trace1 = go.Scatter(
        name =title,
        x=data['Date'],
        y=data[y_axis],
    )
    trace2 = go.Scatter(
        name =title,
        x=data['Date'][data['signals']==1],
        y=data['Close'][data['signals']==1],
        mode='markers',marker_symbol='triangle-up',
        marker_color='white'
    )
    trace3 = go.Scatter(
        name =title,
        x=data['Date'][data['signals']==-1],
        y=data['Close'][data['signals']==-1],
        mode='markers',marker_symbol='triangle-down',
        marker_color='black'
    )
    fig.append_trace(trace1, row=row, col=col)
    fig.append_trace(trace2, row=row, col=col)
    fig.append_trace(trace3, row=row, col=col)
    
    return fig


# symbols_and_names = {'SAMP':"Sampath",'JKH':"Jhon Keels",'DIPD':"DIPD",'SOCL':"Softlogic Capitals"
#                      ,'BROW':"BIL",'HNB':"HNB",'ABAN':"Abans Electricals PLC",'AFSL':"Abans Finance PLC"
#                      ,'ABLT':"Amana Bank Ltd","CHL":"Ceylon Hospitals","CPRT":"Ceylon Printers",
#                     'HUNA':'Hunnas','ACCE':'Access Engineering','ACL':'ACL Cables','APLA':'ACL Plastics',
#                      'ACME':'ACME Printing','AGAL':'Agalawatte Plantations','AGST':'Agstar','AHUN':'Aitken Spence Hotels',
#                      'SPEN':'Aitken Spence PLC','ALLI':'Alliance Finance','ALUM':'Alumex PLC','ATL':'Amana Takaful',
#                      'CONN':'Amaya Leisure','ABEO':'Ambeon Holdings','ANIL':'Anilana Hotels','ARPI':'Arpico Finance','ARIC':'Arpico Insurance',
#                      'ASAS':'Asia Asset Finance','ASIR':'Asiri Hospital','ASSO':'Associated Motor','BFL':'Bairaha Farms','BALA':'Balangoda Plantations',
#                      'BERU':'Beruwala Resorts','BLUE':'Blue Diamonds','BOGA':'Bogala Graphite','BOPL':'Bogawantalawa Tea Estates','BRWN':'Brown & Company',
#                      'BBH':'Browns Beach Hotels','BUKI':'Bukit Darah','CTHR':'C T Holdings','CWM':'C W Mackie','CARG':'Cargills PLC','CABO':'Cargo Boat Development',
#                      'CARS':'Carson Cumberbatch','CFIN':'Central Finance','CIND':'Central Industries','CINS':'Ceylinco Insurance','BREW':'Ceylon Beverage',
#                      'CCS':'Ceylon Cold','GRAN':'Ceylon Grain Elevators','CHL':'Ceylon Hospitals','CHOT':'Ceylon Hotels Corporation','GUAR':'Ceylon Guardian Inv',
#                      'CINV':'Ceylon Investment','CEYL':'Ceylon Tea Brokers','CHMX':'Chemanex PLC','LLUB':'CHEVRON LUBRICANTS','CIC':'CIC','CDBF':'Citizens Development Bus',
#                      'REEF':'Citrus Leisure','PHAR':'Colombo City','DOCK':'Colombo Dockyard','CFI':'Colombo Fort Inv','CIT':'Colombo Investment','COMB':'Commercial Bank',
#                      'COMN':'Commercial Cred & Fina','COMD':'Commercial Development',''
#                      }

stocks_df = investpy.get_stocks(country='Sri Lanka')
symbols_and_names={}
for index, row in stocks_df.iterrows():
    symbols_and_names[row['symbol']]=row['name']

number_of_months = 6
rows = 60
cols = 5
height=3000*4
width=1600
symbols = list(symbols_and_names.keys())
symbols_names = list(symbols_and_names.values())
to_ = date.today().strftime("%d/%m/%Y")
delta = relativedelta.relativedelta(months=number_of_months)
from_ = (date.today() - delta).strftime("%d/%m/%Y")
sym_data_frame = {}
symbols_total_volume = []
for sym in symbols:
    try:
      df = investpy.get_stock_historical_data(stock=sym,
                                          country='Sri Lanka',
                                          from_date=from_,
                                          to_date=to_)
      df = df.reset_index()
      sym_data_frame[sym]=signal_generation(df,parabolic_sar)
      symbols_total_volume.append((sym,sum(df['Volume'])))
    except IndexError as e:
      continue
    


sorted_symbols_total_volume= sorted(symbols_total_volume, key=lambda tup: tup[1],reverse=True) 

sorted_sym_data_frame = {pair[0]: sym_data_frame[pair[0]] for pair in sorted_symbols_total_volume}


fig = make_subplots(rows=rows, cols=cols,subplot_titles=[symbols_and_names[symb] for symb in sorted_sym_data_frame.keys()])
index = 0
for k,v in sorted_sym_data_frame.items():
    row = math.floor(index/cols)+1
    col = index%cols+1
    draw_line_plot(v,k,"Close",row,col,fig)
    index +=1
 
    
fig.update_layout(
        showlegend=False,
        xaxis=dict(
          showticklabels= True
        ),
        height=height, 
        width=width,)  
if number_of_months>2:
  fig.update_xaxes(dtick = 'M1',tickangle = 25)
else:
  fig.update_xaxes(tickangle = 25)
fig = go.Figure(fig)
fig.show()